A copy from https://github.com/Heumi/BEGAN-tensorflow/tree/master/src/function

To test the difference

http://www.foldl.me/uploads/2015/conditional-gans-face-generation/paper.pdf

https://github.com/nightrome/really-awesome-gan

https://arxiv.org/pdf/1705.09966.pdf

https://arxiv.org/pdf/1703.05192.pdf

https://arxiv.org/pdf/1606.03657.pdf

https://arxiv.org/pdf/1704.02166.pdf

https://arxiv.org/pdf/1709.03842.pdf

In [1]:
import os
import scipy.misc as scm
import numpy as np

def make_project_dir(project_dir):
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)
        os.makedirs(os.path.join(project_dir, 'models'))
        os.makedirs(os.path.join(project_dir, 'result'))
        os.makedirs(os.path.join(project_dir, 'result_test'))


def get_image(img_path):
    img = scm.imread(img_path)/255. - 0.5
    img = img[..., ::-1]  # rgb to bgr
    return img

def get_label(path, size):
    label = int(path[-5])
    one_hot = np.zeros(size)
    one_hot[ label ] = 1.
    return one_hot

def inverse_image(img):
    img = (img + 0.5) * 255.
    img[img > 255] = 255
    img[img < 0] = 0
    img = img[..., ::-1] # bgr to rgb
    return img

def pair_expressions(paths):
    subject_exprs = []
    subject_pairs = []
    all_pairs = []
    last_subject = 0

    # Pair all expression of a subject
    for path in paths:
        subject = int(path[-9:-6])

        if subject != last_subject and last_subject != 0:
            subject_pairs = [(x, y) for x in subject_exprs for y in subject_exprs]
            all_pairs.extend(subject_pairs)
            subject_exprs = []

        subject_exprs.append(path)
        last_subject = subject

    # Last subject
    subject_pairs = [(x, y) for x in subject_exprs for y in subject_exprs]
    all_pairs.extend(subject_pairs)
    return all_pairs

def get_shape_c(tensor): # static shape
    return tensor.get_shape().as_list()


In [2]:
import tensorflow as tf
import numpy as np


def conv2d(x, filter_shape, bias=True, stride=1, padding="SAME", name="conv2d"):
    kw, kh, nin, nout = filter_shape
    pad_size = (kw - 1) / 2

    if padding == "VALID":
        x = tf.pad(x, [[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], "SYMMETRIC")

    #initializer = tf.random_normal_initializer(0., 0.02)
    initializer = tf.truncated_normal_initializer(stddev=0.02)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=filter_shape, initializer=initializer)
        x = tf.nn.conv2d(x, weight, [1, stride, stride, 1], padding=padding)

        if bias:
            b = tf.get_variable("bias", shape=filter_shape[-1], initializer=tf.constant_initializer(0.))
            x = tf.nn.bias_add(x, b)
    return x


def fc(x, output_shape, bias=True, name='fc'):
    shape = x.get_shape().as_list()
    dim = np.prod(shape[1:])
    x = tf.reshape(x, [-1, dim])
    input_shape = dim

    #initializer = tf.random_normal_initializer(0., 0.02)
    initializer = tf.truncated_normal_initializer(stddev=0.02)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=[input_shape, output_shape], initializer=initializer)
        x = tf.matmul(x, weight)

        if bias:
            b = tf.get_variable("bias", shape=[output_shape], initializer=tf.constant_initializer(0.))
            x = tf.nn.bias_add(x, b)
    return x


def pool(x, r=2, s=1):
    return tf.nn.avg_pool(x, ksize=[1, r, r, 1], strides=[1, s, s, 1], padding="SAME")


def l1_loss(x, y):
    return tf.reduce_mean(tf.abs(x - y))


def resize_nn(x, size):
    return tf.image.resize_nearest_neighbor(x, size=(int(size), int(size)))

In [3]:
import glob
import time
import numpy as np
import tensorflow as tf

class op_base:
    def __init__(self, sess, project_name):
        self.sess = sess

        # Train
        self.flag = True #args.flag
        self.gpu_number = 0 #args.gpu_number
        self.project = project_name #"test_began" #args.project

        # Train Data
        self.data_dir = "./Face_data/Faces_with_expression_label/dataset_64x64" #args.data_dir #./Data
        self.dataset = "expr" #args.dataset  # celeba
        self.data_size = 64 #args.data_size  # 64 or 128
        self.data_opt = "crop" #args.data_opt  # raw or crop
        self.data_label_vector_size = 7 #size of one-hot-encoded label vector

        # Train Iteration
        self.niter = 500 #50 #args.niter
        self.niter_snapshot = 200 #2440 #args.nsnapshot
        self.max_to_keep = 5 #args.max_to_keep

        # Train Parameter
        self.batch_size = 32 #16 #args.batch_size
        self.learning_rate =  2e-4 # <----WGAN #1e-4  <-- BEGAN #args.learning_rate
        self.mm = 0.5 #args.momentum
        self.mm2 = 0.999 #args.momentum2
        self.lamda = 0.001 #args.lamda
        self.gamma = 2.0 #args.gamma
        self.filter_number = 64 #args.filter_number
        self.input_size = 64 #args.input_size
        self.embedding = 64 #args.embedding

        # Result Dir & File
        self.project_dir = 'assets/{0}_{1}_{2}_{3}/'.format(self.project, self.dataset, self.data_opt, self.data_size)
        self.ckpt_dir = os.path.join(self.project_dir, 'models')
        self.model_name = "{0}.model".format(self.project)
        self.ckpt_model_name = os.path.join(self.ckpt_dir, self.model_name)

        # etc.
        if not os.path.exists('assets'):
            os.makedirs('assets')
        make_project_dir(self.project_dir)

    def load(self, sess, saver, ckpt_dir):
        ckpt = tf.train.get_checkpoint_state(ckpt_dir)
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(ckpt_dir, ckpt_name))

In [4]:
import glob
import time
import datetime


class Operator(op_base):
    def __init__(self, sess, project_name):
        op_base.__init__(self, sess, project_name)
        self.build_model()

    def build_model(self):
        # Input placeholder
        self.x = tf.placeholder(tf.float32, shape=[None, self.input_size, self.input_size, 3], name='x')
        self.y = tf.placeholder(tf.float32, shape=[None, self.data_size, self.data_size, 3], name='y')
        self.c = tf.placeholder(tf.float32, shape=[None, self.data_label_vector_size], name='c')
        self.lr = tf.placeholder(tf.float32, name='lr')

        # Generator
        self.recon_gen = self.generator_decoder(self.generator_encoder(self.x), self.c)

        # Discriminator (Critic)
        d_real = self.discriminator(self.y, self.c)
        d_fake = self.discriminator(self.recon_gen, self.c, reuse=True)

        # Loss --------------------------------------------------
        self.d_loss = -tf.reduce_mean(tf.log(d_real) + tf.log(1.-d_fake))
        self.g_loss = -tf.reduce_mean(tf.log(d_fake))
        #--------------------------------------------------------
        
        
        # Variables
        g_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "gen_")
        d_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "disc_")

        
        # Optimizer ------------------------------------------------
        self.opt_d = tf.train.AdamOptimizer(learning_rate=self.lr, beta1=0.5).minimize(self.d_loss, var_list=d_vars)
        self.opt_g = tf.train.AdamOptimizer(learning_rate=self.lr, beta1=0.5).minimize(self.g_loss, var_list=g_vars)
        #-----------------------------------------------------------


        # initializer
        self.sess.run(tf.global_variables_initializer())

        # tf saver
        self.saver = tf.train.Saver(max_to_keep=(self.max_to_keep))

        try:
            self.load(self.sess, self.saver, self.ckpt_dir)
        except:
            # save full graph
            self.saver.save(self.sess, self.ckpt_model_name, write_meta_graph=True)

        # Summary
        if self.flag:
            tf.summary.scalar('loss/loss', self.d_loss + self.g_loss)
            tf.summary.scalar('loss/g_loss', self.g_loss)
            tf.summary.scalar('loss/d_loss', self.d_loss)
            self.merged = tf.summary.merge_all()
            self.writer = tf.summary.FileWriter(self.project_dir, self.sess.graph)

    def train(self, train_flag):
        # load data
        data_path = self.data_dir

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            data = pair_expressions(data)
            np.save(data_path + '.npy', data)

        # initial parameter
        start_time = time.time()
        lr = np.float32(self.learning_rate)
        self.count = 0

        for epoch in range(self.niter):
            random_order = np.random.permutation(len(data))
            data = [data[i] for i in random_order[:]]
            
            batch_idxs = len(data) // self.batch_size

            for idx in range(0, batch_idxs):
                self.count += 1

                batch_files = data[idx * self.batch_size: (idx + 1) * self.batch_size]
                batch_inputs = [get_image(batch_file[0]) for batch_file in batch_files]
                batch_targets = [get_image(batch_file[1]) for batch_file in batch_files]
                batch_target_labels = [get_label(batch_file[1], self.data_label_vector_size) for batch_file in batch_files]

                d_opt = [self.opt_d, self.d_loss, self.merged]
                g_opt = [self.opt_g, self.g_loss]
                
                feed_dict_d = {self.x: batch_inputs, self.y: batch_targets, self.c: batch_target_labels,
                               self.lr: lr}
                feed_dict_g = {self.x: batch_inputs, self.c: batch_target_labels, self.lr: lr}

                _, loss_d, summary = self.sess.run(d_opt, feed_dict=feed_dict_d)
                _, loss_g = self.sess.run(g_opt, feed_dict=feed_dict_g) #Update the generator, twice for good measure.
                _, loss_g = self.sess.run(g_opt, feed_dict=feed_dict_g)
                    
                if self.count % 5 == 1:
                    print("Epoch: [%2d] [%4d/%4d] time: %4.4f, "
                          "loss_g: %.4f, loss_d: %.4f"
                          % (epoch, idx, batch_idxs, time.time() - start_time,
                             loss_g, loss_d))

                # write train summary
                self.writer.add_summary(summary, self.count)

                # Test during Training
                if self.count % self.niter_snapshot == (self.niter_snapshot - 1):                    
                    # save & test
                    self.saver.save(self.sess, self.ckpt_model_name, global_step=self.count, write_meta_graph=False)
                    self.test(train_flag)

    def test(self, train_flag=True):
        # generate output
        img_num =  36 #self.batch_size
        display_img_num = int(img_num / 3)
        img_size = self.data_size

        output_f = int(np.sqrt(img_num))
        im_output_gen = np.zeros([img_size * output_f, img_size * output_f, 3])
        
        # load data
        data_path = self.data_dir

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            data = pair_expressions(data)
            np.save(data_path + '.npy', data)

        print('Test data shuffle ....')
        random_order = np.random.permutation(len(data))
        data = [data[i] for i in random_order[:]]
        print('Test data shuffle Done')
        
        batch_files = data[0: display_img_num]
        test_inputs = [get_image(batch_file[0]) for batch_file in batch_files]
        test_inputs_o = [scm.imread((batch_file[0])) for batch_file in batch_files]
        test_targets = [scm.imread((batch_file[1])) for batch_file in batch_files]
        test_target_labels = [get_label(batch_file[1], self.data_label_vector_size) for batch_file in batch_files]

        output_gen = (self.sess.run(self.recon_gen, feed_dict={self.x: test_inputs, 
                                                               self.c: test_target_labels}))  # generator output

        output_gen = [inverse_image(output_gen[i]) for i in range(display_img_num)]

        for i in range(output_f): # row
            for j in range(output_f): # col
                if j % 3 == 0: # input img
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = test_inputs_o[int(j / 3) + (i * int(output_f / 3))]
                elif j % 3 == 1: # output img
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = output_gen[int(j / 3) + (i * int(output_f / 3))]
                else: # target img
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = test_targets[int(j / 3) + (i * int(output_f / 3))]

        labels = np.argmax(test_target_labels, axis=1)
        label_string = ''.join(str(int(l)) for l in labels)
        # output save
        if train_flag:
            scm.imsave(self.project_dir + '/result/' + str(self.count) + '_' + label_string 
                       + '_output.bmp', im_output_gen)
        else:
            now = datetime.datetime.now()
            nowDatetime = now.strftime('%Y-%m-%d_%H:%M:%S')
            scm.imsave(self.project_dir + '/result_test/gen_{}_'.format(nowDatetime) 
                       + label_string + '_output.bmp', im_output_gen)


In [ ]:

class CWGAN(Operator):
    def __init__(self, sess, project_name):
        Operator.__init__(self, sess, project_name)
        
    def generator_encoder(self, x, reuse=None):
        with tf.variable_scope('gen_') as scope:
            if reuse:
                scope.reuse_variables()

            f = self.filter_number
            h = self.embedding
            p = "SAME"

            x = conv2d(x, [3, 3, 3, f], stride=1,  padding=p,name='conv1_enc_a')
            x = tf.nn.elu(x)

            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv2_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv2_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, f, 2 * f], stride=1,  padding=p,name='conv3_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 2 * f, 2 * f], stride=1,  padding=p,name='conv3_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 2 * f, 2 * f], stride=1,  padding=p,name='conv3_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, 2 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 3 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 3 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, 3 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 4 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 4 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_b')
            x = tf.nn.elu(x)

            if self.data_size == 128:
                x = conv2d(x, [1, 1, 4 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_0')
                x = pool(x, r=2, s=2)
                x = conv2d(x, [3, 3, 5 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_a')
                x = tf.nn.elu(x)
                x = conv2d(x, [3, 3, 5 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_b')
                x = tf.nn.elu(x)

            x = fc(x, h, name='enc_fc')
        return x

    def generator_decoder(self, x, c, reuse=None):
        with tf.variable_scope('gen_') as scope:
            if reuse:
                scope.reuse_variables()

            w = self.data_size
            f = self.filter_number
            p = "SAME"
            
            x = tf.concat([x, c], axis=1) # adding label information

            x = fc(x, 8 * 8 * f, name='fc')
            x = tf.reshape(x, [-1, 8, 8, f])

            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv1_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv1_b')
            x = tf.nn.elu(x)

            if self.data_size == 128:
                x = resize_nn(x, w / 8)
                x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv2_a')
                x = tf.nn.elu(x)
                x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv2_b')
                x = tf.nn.elu(x)

            x = resize_nn(x, w / 4)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv3_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv3_b')
            x = tf.nn.elu(x)

            x = resize_nn(x, w / 2)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv4_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv4_b')
            x = tf.nn.elu(x)

            x = resize_nn(x, w)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv5_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv5_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [3, 3, f, 3], stride=1,  padding=p,name='conv6_a')
        return x

    def discriminator(self, x, c, reuse=None):
        with tf.variable_scope('disc_') as scope:
            if reuse:
                scope.reuse_variables()

            f = self.filter_number
            h = self.embedding
            p = "SAME"
            c_len = self.data_label_vector_size

            x = conv2d(x, [3, 3, 3, f], stride=1,  padding=p,name='conv1_enc_a')
            x = tf.nn.elu(x)
            
            # Add y as a channel to dis1 layer as described in IcGAN paper
            x = tf.concat([x, tf.tile(tf.reshape(c, [-1, 1, 1, get_shape_c(c)[-1]]),\
                                     [1, x.get_shape().as_list()[1], x.get_shape().as_list()[2], 1])],\
                              axis=3)

            x = conv2d(x, [3, 3, f + c_len, f], stride=1,  padding=p,name='conv2_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv2_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, f, 2 * f], stride=1,  padding=p,name='conv3_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 2 * f, 2 * f], stride=1,  padding=p,name='conv3_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 2 * f, 2 * f], stride=1,  padding=p,name='conv3_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, 2 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 3 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 3 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, 3 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 4 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 4 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_b')
            x = tf.nn.elu(x)

            if self.data_size == 128:
                x = conv2d(x, [1, 1, 4 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_0')
                x = pool(x, r=2, s=2)
                x = conv2d(x, [3, 3, 5 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_a')
                x = tf.nn.elu(x)
                x = conv2d(x, [3, 3, 5 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_b')
                x = tf.nn.elu(x)

            x = fc(x, 1, name='enc_fc')
            x = tf.nn.sigmoid(x)
        return x


In [ ]:
import distutils.util
import os
import tensorflow as tf

''' config settings '''

project_name = "CDCGAN_exprs_1"
train_flag = True

'''-----------------'''

gpu_number = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #args.gpu_number

with tf.device('/gpu:{0}'.format(gpu_number)):
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.90)
    config = tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options)

    with tf.Session(config=config) as sess:
        model = CWGAN(sess, project_name)

        # TRAIN / TEST
        if train_flag:
            model.train(train_flag)
        else:
            model.test(train_flag)

Epoch: [ 0] [   0/ 316] time: 1.2658, loss_g: 0.6880, loss_d: 1.3863
Epoch: [ 0] [   5/ 316] time: 2.9417, loss_g: 0.5790, loss_d: 1.3552
Epoch: [ 0] [  10/ 316] time: 4.6155, loss_g: 0.7247, loss_d: 1.3931
Epoch: [ 0] [  15/ 316] time: 6.2909, loss_g: 0.7626, loss_d: 1.3576
Epoch: [ 0] [  20/ 316] time: 7.9682, loss_g: 0.6680, loss_d: 1.4034
Epoch: [ 0] [  25/ 316] time: 9.6447, loss_g: 0.7047, loss_d: 1.4177
Epoch: [ 0] [  30/ 316] time: 11.3214, loss_g: 0.7098, loss_d: 1.3845
Epoch: [ 0] [  35/ 316] time: 13.0072, loss_g: 0.7080, loss_d: 1.3838
Epoch: [ 0] [  40/ 316] time: 14.6911, loss_g: 0.7067, loss_d: 1.3829
Epoch: [ 0] [  45/ 316] time: 16.3727, loss_g: 0.7048, loss_d: 1.3821
Epoch: [ 0] [  50/ 316] time: 18.0566, loss_g: 0.7003, loss_d: 1.3829
Epoch: [ 0] [  55/ 316] time: 19.7408, loss_g: 0.6938, loss_d: 1.3862
Epoch: [ 0] [  60/ 316] time: 21.4285, loss_g: 0.6866, loss_d: 1.4007
Epoch: [ 0] [  65/ 316] time: 23.1183, loss_g: 0.6957, loss_d: 1.3868
Epoch: [ 0] [  70/ 316] ti

Epoch: [ 1] [ 259/ 316] time: 202.7015, loss_g: 0.6926, loss_d: 1.3869
Epoch: [ 1] [ 264/ 316] time: 204.4897, loss_g: 0.6927, loss_d: 1.3866
Epoch: [ 1] [ 269/ 316] time: 206.2709, loss_g: 0.6928, loss_d: 1.3864
Epoch: [ 1] [ 274/ 316] time: 208.0503, loss_g: 0.6929, loss_d: 1.3863
Epoch: [ 1] [ 279/ 316] time: 209.8327, loss_g: 0.6929, loss_d: 1.3862
Test data shuffle ....
Test data shuffle Done
Epoch: [ 1] [ 284/ 316] time: 211.6888, loss_g: 0.6930, loss_d: 1.3861
Epoch: [ 1] [ 289/ 316] time: 213.4694, loss_g: 0.6931, loss_d: 1.3859
Epoch: [ 1] [ 294/ 316] time: 215.2499, loss_g: 0.6932, loss_d: 1.3857
Epoch: [ 1] [ 299/ 316] time: 217.0293, loss_g: 0.6929, loss_d: 1.3862
Epoch: [ 1] [ 304/ 316] time: 218.8102, loss_g: 0.6934, loss_d: 1.3871
Epoch: [ 1] [ 309/ 316] time: 220.5923, loss_g: 0.6935, loss_d: 1.3866
Epoch: [ 1] [ 314/ 316] time: 222.3745, loss_g: 0.6932, loss_d: 1.3866
Epoch: [ 2] [   3/ 316] time: 224.1546, loss_g: 0.6934, loss_d: 1.3864
Epoch: [ 2] [   8/ 316] time: 2

Epoch: [ 3] [ 197/ 316] time: 405.9696, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 3] [ 202/ 316] time: 407.7583, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 3] [ 207/ 316] time: 409.5381, loss_g: 0.6931, loss_d: 1.3864
Epoch: [ 3] [ 212/ 316] time: 411.3185, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 3] [ 217/ 316] time: 413.0995, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 3] [ 222/ 316] time: 414.8809, loss_g: 0.6930, loss_d: 1.3863
Epoch: [ 3] [ 227/ 316] time: 416.6617, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 3] [ 232/ 316] time: 418.4420, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 3] [ 237/ 316] time: 420.2215, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 3] [ 242/ 316] time: 422.0022, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 3] [ 247/ 316] time: 423.7831, loss_g: 0.6931, loss_d: 1.3863
Test data shuffle ....
Test data shuffle Done
Epoch: [ 3] [ 252/ 316] time: 425.6515, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 3] [ 257/ 316] time: 427.4323, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 3] [ 262/ 316] time: 4

Epoch: [ 5] [ 135/ 316] time: 609.2855, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 5] [ 140/ 316] time: 611.0677, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 5] [ 145/ 316] time: 612.8494, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 5] [ 150/ 316] time: 614.6303, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 5] [ 155/ 316] time: 616.4102, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 5] [ 160/ 316] time: 618.1908, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 5] [ 165/ 316] time: 619.9729, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 5] [ 170/ 316] time: 621.7572, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 5] [ 175/ 316] time: 623.5401, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 5] [ 180/ 316] time: 625.3208, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 5] [ 185/ 316] time: 627.1017, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 5] [ 190/ 316] time: 628.8832, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 5] [ 195/ 316] time: 630.6660, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 5] [ 200/ 316] time: 632.4475, loss_g: 0.6932, loss_d: 1.3863
Epoch:

Epoch: [ 7] [  73/ 316] time: 812.6271, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 7] [  78/ 316] time: 814.4091, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 7] [  83/ 316] time: 816.1892, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 7] [  88/ 316] time: 817.9708, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 7] [  93/ 316] time: 819.7537, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 7] [  98/ 316] time: 821.5370, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 7] [ 103/ 316] time: 823.3186, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 7] [ 108/ 316] time: 825.0997, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 7] [ 113/ 316] time: 826.8803, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 7] [ 118/ 316] time: 828.6629, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 7] [ 123/ 316] time: 830.4452, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 7] [ 128/ 316] time: 832.2262, loss_g: 0.6932, loss_d: 1.3863
Epoch: [ 7] [ 133/ 316] time: 834.0064, loss_g: 0.6931, loss_d: 1.3863
Epoch: [ 7] [ 138/ 316] time: 835.7885, loss_g: 0.6931, loss_d: 1.3863
Epoch:

...

Epoch: [19] [ 216/ 316] time: 2216.4323, loss_g: 0.6932, loss_d: 1.3863
Epoch: [19] [ 221/ 316] time: 2218.2137, loss_g: 0.6932, loss_d: 1.3863
Epoch: [19] [ 226/ 316] time: 2219.9949, loss_g: 0.6932, loss_d: 1.3863
Epoch: [19] [ 231/ 316] time: 2221.7750, loss_g: 0.6931, loss_d: 1.3863
Epoch: [19] [ 236/ 316] time: 2223.5555, loss_g: 0.6931, loss_d: 1.3863
Epoch: [19] [ 241/ 316] time: 2225.3359, loss_g: 0.6932, loss_d: 1.3863
Epoch: [19] [ 246/ 316] time: 2227.1167, loss_g: 0.6931, loss_d: 1.3863
Epoch: [19] [ 251/ 316] time: 2228.8982, loss_g: 0.6932, loss_d: 1.3863
Epoch: [19] [ 256/ 316] time: 2230.6804, loss_g: 0.6931, loss_d: 1.3863
Epoch: [19] [ 261/ 316] time: 2232.4617, loss_g: 0.6931, loss_d: 1.3863
Epoch: [19] [ 266/ 316] time: 2234.2432, loss_g: 0.6932, loss_d: 1.3863
Epoch: [19] [ 271/ 316] time: 2236.0279, loss_g: 0.6929, loss_d: 1.3863
Epoch: [19] [ 276/ 316] time: 2237.8099, loss_g: 0.6935, loss_d: 1.3863
Epoch: [19] [ 281/ 316] time: 2239.5928, loss_g: 0.6924, loss_d:

Epoch: [21] [ 149/ 316] time: 2417.9212, loss_g: 0.6932, loss_d: 1.3863
Epoch: [21] [ 154/ 316] time: 2419.7018, loss_g: 0.6933, loss_d: 1.3863
Epoch: [21] [ 159/ 316] time: 2421.4871, loss_g: 0.6931, loss_d: 1.3863
Test data shuffle ....
Test data shuffle Done
Epoch: [21] [ 164/ 316] time: 2423.3482, loss_g: 0.6932, loss_d: 1.3863
Epoch: [21] [ 169/ 316] time: 2425.1295, loss_g: 0.6932, loss_d: 1.3863
Epoch: [21] [ 174/ 316] time: 2426.9109, loss_g: 0.6931, loss_d: 1.3863
Epoch: [21] [ 179/ 316] time: 2428.6926, loss_g: 0.6932, loss_d: 1.3863
Epoch: [21] [ 184/ 316] time: 2430.4721, loss_g: 0.6931, loss_d: 1.3863
Epoch: [21] [ 189/ 316] time: 2432.2537, loss_g: 0.6932, loss_d: 1.3863
Epoch: [21] [ 194/ 316] time: 2434.0337, loss_g: 0.6930, loss_d: 1.3863
Epoch: [21] [ 199/ 316] time: 2435.8146, loss_g: 0.6934, loss_d: 1.3863
Epoch: [21] [ 204/ 316] time: 2437.5962, loss_g: 0.6929, loss_d: 1.3863
Epoch: [21] [ 209/ 316] time: 2439.3793, loss_g: 0.6935, loss_d: 1.3863
Epoch: [21] [ 214/

Epoch: [23] [  77/ 316] time: 2617.6475, loss_g: 0.6931, loss_d: 1.3863
Epoch: [23] [  82/ 316] time: 2619.4284, loss_g: 0.6932, loss_d: 1.3863
Epoch: [23] [  87/ 316] time: 2621.2100, loss_g: 0.6931, loss_d: 1.3863
Epoch: [23] [  92/ 316] time: 2622.9924, loss_g: 0.6932, loss_d: 1.3863
Epoch: [23] [  97/ 316] time: 2624.7731, loss_g: 0.6932, loss_d: 1.3863
Epoch: [23] [ 102/ 316] time: 2626.5530, loss_g: 0.6931, loss_d: 1.3863
Epoch: [23] [ 107/ 316] time: 2628.3360, loss_g: 0.6932, loss_d: 1.3863
Epoch: [23] [ 112/ 316] time: 2630.1191, loss_g: 0.6929, loss_d: 1.3863
Epoch: [23] [ 117/ 316] time: 2631.9016, loss_g: 0.6934, loss_d: 1.3863
Epoch: [23] [ 122/ 316] time: 2633.6829, loss_g: 0.6927, loss_d: 1.3863
Epoch: [23] [ 127/ 316] time: 2635.4643, loss_g: 0.6939, loss_d: 1.3863
Test data shuffle ....
Test data shuffle Done
Epoch: [23] [ 132/ 316] time: 2637.3257, loss_g: 0.6922, loss_d: 1.3863
Epoch: [23] [ 137/ 316] time: 2639.1080, loss_g: 0.6940, loss_d: 1.3863
Epoch: [23] [ 142/

Epoch: [25] [   5/ 316] time: 2817.4300, loss_g: 0.6931, loss_d: 1.3863
Epoch: [25] [  10/ 316] time: 2819.2119, loss_g: 0.6932, loss_d: 1.3863
Epoch: [25] [  15/ 316] time: 2820.9928, loss_g: 0.6932, loss_d: 1.3863
Epoch: [25] [  20/ 316] time: 2822.7741, loss_g: 0.6931, loss_d: 1.3863
Epoch: [25] [  25/ 316] time: 2824.5567, loss_g: 0.6932, loss_d: 1.3863
Epoch: [25] [  30/ 316] time: 2826.3393, loss_g: 0.6931, loss_d: 1.3863
Epoch: [25] [  35/ 316] time: 2828.1211, loss_g: 0.6931, loss_d: 1.3863
Epoch: [25] [  40/ 316] time: 2829.9041, loss_g: 0.6932, loss_d: 1.3863
Epoch: [25] [  45/ 316] time: 2831.6846, loss_g: 0.6932, loss_d: 1.3863
Epoch: [25] [  50/ 316] time: 2833.4666, loss_g: 0.6931, loss_d: 1.3863
Epoch: [25] [  55/ 316] time: 2835.2487, loss_g: 0.6933, loss_d: 1.3863
Epoch: [25] [  60/ 316] time: 2837.0330, loss_g: 0.6930, loss_d: 1.3863
Epoch: [25] [  65/ 316] time: 2838.8169, loss_g: 0.6934, loss_d: 1.3863
Epoch: [25] [  70/ 316] time: 2840.5987, loss_g: 0.6928, loss_d:

Epoch: [26] [ 249/ 316] time: 3017.2597, loss_g: 0.6930, loss_d: 1.3863
Epoch: [26] [ 254/ 316] time: 3019.0439, loss_g: 0.6934, loss_d: 1.3863
Epoch: [26] [ 259/ 316] time: 3020.8283, loss_g: 0.6928, loss_d: 1.3863
Epoch: [26] [ 264/ 316] time: 3022.6098, loss_g: 0.6937, loss_d: 1.3863
Epoch: [26] [ 269/ 316] time: 3024.3914, loss_g: 0.6925, loss_d: 1.3863
Epoch: [26] [ 274/ 316] time: 3026.1732, loss_g: 0.6939, loss_d: 1.3863
Epoch: [26] [ 279/ 316] time: 3027.9562, loss_g: 0.6925, loss_d: 1.3863
Epoch: [26] [ 284/ 316] time: 3029.7375, loss_g: 0.6935, loss_d: 1.3863
Epoch: [26] [ 289/ 316] time: 3031.5193, loss_g: 0.6929, loss_d: 1.3863
Epoch: [26] [ 294/ 316] time: 3033.3016, loss_g: 0.6932, loss_d: 1.3863
Epoch: [26] [ 299/ 316] time: 3035.0839, loss_g: 0.6931, loss_d: 1.3863
Epoch: [26] [ 304/ 316] time: 3036.8661, loss_g: 0.6931, loss_d: 1.3863
Epoch: [26] [ 309/ 316] time: 3038.6476, loss_g: 0.6932, loss_d: 1.3863
Epoch: [26] [ 314/ 316] time: 3040.4281, loss_g: 0.6932, loss_d: